<a href="https://colab.research.google.com/github/TTReza/Pytorch_Knowledge/blob/master/pytorchnnImplement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataloader
from torchvision import datasets
from torchvision.transforms import ToTensor

ImportError: cannot import name 'Dataloader' from 'torch.utils.data' (/usr/local/lib/python3.10/dist-packages/torch/utils/data/__init__.py)

In [2]:
!pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.6 MB/s eta 0:00:00


In [4]:
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
train = datasets.MNIST(root= 'data', download= True, train=True, transform=ToTensor())
dataset = DataLoader(train,32)

100%|██████████| 9912422/9912422 [00:00<00:00, 335761851.36it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 27995307.10it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 115316749.15it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 13925825.12it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [7]:
from torch import nn, load, save
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1, 32, (3,3)),
        nn.ReLU(),
        nn.Conv2d(32, 64, (3,3)),
        nn.ReLU(),
        nn.Conv2d(64, 64, (3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6),10)

    )
  def forward(self, x):
    return self.model(x)


clf = ImageClassifier().to('cpu')
opt = Adam(clf.parameters(), lr =1e-3)
loss_fn = nn.CrossEntropyLoss()

#training flow
if __name__ == "__main__":
  for epoch in range(10):
    for batch in dataset:
      X, y = batch
      X, y = X.to('cpu'), y.to('cpu')
      yhat = clf(X)
      loss = loss_fn(yhat, y)

      #apply backpropagation
      opt.zero_grad()
      loss.backward()
      opt.step()

    print(f"Epoch:{epoch} loss is {loss.item()}")
  with open('model_state.pt', 'wb') as f:
    save(clf.state_dict(), f)

Epoch:0 loss is 0.01093566045165062
Epoch:1 loss is 0.00513667706400156
Epoch:2 loss is 0.0012688657734543085
Epoch:3 loss is 0.016673393547534943
Epoch:4 loss is 4.3916210415773094e-05
Epoch:5 loss is 9.151804260909557e-05
Epoch:6 loss is 4.38048446085304e-05
Epoch:7 loss is 6.142405709397281e-06
Epoch:8 loss is 8.401995728490874e-05
Epoch:9 loss is 2.4772327833488816e-06


In [12]:
import torch
from PIL import Image
import numpy as np
if __name__ == "__main__":
  with open('model_state.pt', 'rb') as f:
    clf.load_state_dict(load(f))

  img = Image.open('img_1.png')
  #img_tensor = ToTensor()(img).unsqueeze(0).to('cpu')
  img_tensor = torch.from_numpy(np.array(img)).float().unsqueeze(0).to('cpu')

  #print(torch.argmax(clf(img_tensor)))
  assert img_tensor.shape == torch.Size([1, 3, 224, 224])
  print(torch.argmax(clf(img_tensor)))

AssertionError: 